In [ ]:
%load_ext autoreload
%autoreload 2

import hydra
import os
import datetime
from pathlib import Path

# Initialize hydra and move to the root of the repository
try:
    hydra.initialize(version_base=None, config_path="../config/")
    CONFIG = hydra.compose(config_name="main.yaml")
    print('Initializing hydra')
except:
    print('Hydra already initalized!')
else:
    os.chdir('..')
    # Create an output folder in the root of the repository
    OUTPUT_FOLDER = Path('output/{0}'.format(datetime.datetime.now()).replace(' ', '_'))
    Path(OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from src.utils.colors import flatuicolors
from src.utils.styling import get_standard_colors
from src.weighted_network import WeightedNetwork
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cf

In [ ]:
def map_setup(ax):
    ax.coastlines()
    ax.add_feature(cf.BORDERS, ls=':')
    ax.add_feature(cf.LAND, color='k', alpha=0.05)

def plot_community(ax, network, comms=None, reference_network=None):

    if not reference_network:
        reference_network = network
        
    colors = get_standard_colors()
    G = network.G()
    pos = network.position()
    G_ref = reference_network.G()

    if not comms:
        comms = network.communities()
    
    nx.draw_networkx_edges(G, pos, alpha=0.01, ax=ax)
    sc = nx.draw_networkx_nodes(G_ref, pos, node_size=8, node_color='w', alpha=0.25, ax=ax)
    sc.set_edgecolor('k')
    
    for i, comm in enumerate(comms):
        sc = nx.draw_networkx_nodes(G, pos, nodelist=comm, node_size=12, node_color=colors[i], ax=ax)
        sc.set_edgecolor('k')
        sc.set_linewidth(.15)

def load(year0=None, year1=None):
    input_path = Path(CONFIG.data.raw)
    nodes_file = input_path / CONFIG.data.filenames.nodes
    edges_file = input_path / CONFIG.data.filenames.edges
    return WeightedNetwork(nodes_file=nodes_file, edges_file=edges_file, year0=year0, year1=year1)

def finalize(filename):
    os.system(f'pdfcrop --margin 5 {filename} {filename}')
    os.system(f'convert -density 400 {filename} {filename}.jpg')

def get_temporal_communities(bins):

    # Get list of communities for each time slice
    comm_list = []
    for year0, year1 in bins:
        G = load(year0=year0, year1=year1)
        comm_list.append(G.communities())

    sorted_comm_list = [comm_list[0]]
    
    for i in range(1, len(comm_list)):
        
        old_comm = sorted_comm_list[i-1]
        new_comm = comm_list[i]
        sorted_new_comm = []
    
        for oc in old_comm:
            overlap = [len(oc.intersection(nc)) for nc in new_comm]
            index = np.argmax(overlap)
            sorted_new_comm.append(new_comm[index])
            new_comm = new_comm[:index] + new_comm[index+1:]
    
        for nc in new_comm:
            sorted_new_comm.append(nc)
    
        sorted_comm_list.append(sorted_new_comm)

    return sorted_comm_list

In [ ]:
G = load()
CRS = ccrs.LambertAzimuthalEqualArea(
    central_latitude=CONFIG.grid.central_latitidue, 
    central_longitude=CONFIG.grid.central_longitude, 
    false_easting=CONFIG.grid.false_easting, 
    false_northing=CONFIG.grid.false_northing
)

In [ ]:
ax = plt.axes(projection=CRS)
map_setup(ax)

plot_community(ax, G)
filename = OUTPUT_FOLDER / 'communities.pdf'
plt.savefig(filename)

finalize(filename)

In [ ]:
# Work in progress. 
# TODO: Don't hardcode number of axes and extent.
def plot_temporal_communities(bins, network, temporal_communites, filename='temporal_communities.pdf', axis_order=None):

    f, axarr = plt.subplots(2, 3, subplot_kw={'projection': CRS}, figsize=(7.5 * 1.25, 8 * 1.25))
    plt.subplots_adjust(wspace=.05, hspace=.1)
    
    axarr = axarr.flatten()
    if axis_order is not None:
        axarr = axarr.flatten()[axis_order]
    
    for bin, ax, comm in zip(bins, axarr, temporal_communites):
        ax.set_title(f'{bin[0]} to {bin[1]}')
        sliced_network = load(year0=bin[0], year1=bin[1])
        map_setup(ax)
        plot_community(ax, sliced_network, comm, reference_network=network)
        ax.set_extent([4, 11.5, 44, 53.5])
    
    filename = OUTPUT_FOLDER / filename
    plt.savefig(filename)
    finalize(filename)

In [ ]:
BINS = [(1351, 1354), (1355, 1400), (1401, 1414), (1415, 1417), (1415, 1417), (1418, 1450)]
comm_list = get_temporal_communities(BINS)
plot_temporal_communities(BINS, G, comm_list, filename='temporal_networks_hamming.pdf', axis_order=[0, 3, 1, 2, 4, 5]) 

In [ ]:
BINS = [(1000, 1100), (1101, 1200), (1201, 1250), (1251, 1300), (1301, 1400), (1400, 1520)]
comm_list = get_temporal_communities(BINS)
plot_temporal_communities(BINS, G, comm_list, filename='temporal_networks.pdf')